In [1]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

# import streamlit as st
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_groq import ChatGroq
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
import json

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_chroma import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.environ["GROQ2_API_KEY"]

In [ ]:
class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """

        with open(self.file_path, encoding="utf-8") as f:
            data=json.load(f)
            for i, j in data.items():
                yield Document(
                    page_content=j,
                    metadata={"topic": i,"source": self.file_path},
                )

In [ ]:
with open("../data/transcriptions.json", "r", encoding="utf-8") as f:
    data2=json.load(f)

loader = CustomDocumentLoader("../data/transcriptions.json")

In [ ]:
#This are the Video's title
for i,j in data2.items():
    print(i)

In [ ]:
documents=loader.load()
documents

In [ ]:
embeddings=OllamaEmbeddings(model="mxbai-embed-large")


text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
vectors=Chroma.from_documents(final_documents,embeddings,persist_directory="./chroma_db")

In [ ]:
db3 = Chroma(persist_directory="./chroma_db",embedding_function=embeddings)

---

# To test this embeddings you can use retriever.

In [ ]:
retriever.invoke("Summarize Unit 10 Topiuc 1")